In [2]:
import azureml

from azureml.core import Run
from azureml.core import Workspace

from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.environment import Environment

from azureml.core.experiment import Experiment

from azureml.core.image import Image

from azureml.core.model import InferenceConfig
from azureml.core.model import Model

from azureml.core.run import Run

from azureml.core.webservice import AciWebservice, Webservice

StatementMeta(spark8, 1, 4, Finished, Available)

In [3]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = b"

#Provide values for the existing Resource Group 
resource_group09-1"

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "Cosmos-DB-6tqjlmi"
workspace_region = "East US"

model_name = "batt-cycles-7"

description = 'Forecast'
cpu_cores = 1
memory_gb = 1
tags = {'name':'scoring'}

conda_packages = ['numpy', 'pandas', 'scikit-learn', 'py-xgboost']
pip_packages = ['inference-schema', 'azureml-train-automl']
conda_file = "dependencies.yml"

env_name = "myenv"
scoring_file = "scoring_service.py"

service_name = "aciservice-3uka6z6tqjlmi"

StatementMeta(spark8, 1, 5, Submitted, Available)

System.NullReferenceException: Object reference not set to an instance of an object.

In [ ]:
# Values from `Setup Service Principal Authentication` in the Batch_Scoring notebook - do that first, then get those values and paste them here
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47" # Use "tenantId" value
service_principal_id = "df87bb49-ae74-40f8-baf8-e7e4474d4ad7" # Use "clientId" value
service_principal_password = "A~GWfa7VAvqJp.9UT5aCGPyiP6LAVVgZkh" # Use "clientSecret" value


StatementMeta(spark8, 1, 6, Submitted, Available)

In [ ]:
from azureml.core.authentication import ServicePrincipalAuthentication

sp = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=service_principal_id,
    service_principal_password=service_principal_password)

StatementMeta(spark8, 1, 7, Finished, Available)

In [ ]:
from azureml.core import Workspace

ws = Workspace.get(
    name=workspace_name, 
    auth=sp,
    subscription_id=subscription_id)

StatementMeta(spark8, 1, 8, Finished, Available)

In [ ]:
ws.get_details()

StatementMeta(spark8, 1, 9, Finished, Available)

In [ ]:
scoring_service = """
import os
import json
import pickle
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

### input_sample = pd.DataFrame(data=[{'battery_Age_Days': 120, 'daily_Trip_Duration': 590}])
input_sample = pd.DataFrame(data=[{"Date":"2013-01-01T00:00:00.000Z","Battery_ID":0,"Battery_Age_Days":0,"Daily_Trip_Duration":67.8456075842}])

def init():
    global model
    # This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = Model.get_model_path(model_name = 'batt-cycles-7')
    model = joblib.load(model_path)

@input_schema('data', PandasParameterType(input_sample))
def run(data):
    try:
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

    return json.dumps({"result": result.tolist()})
"""

with open(scoring_file, "w") as file:
  file.write(scoring_service)

StatementMeta(spark8, 1, 10, Submitted, Available)

In [ ]:
registeredModel = Model(workspace = ws, name = model_name)

StatementMeta(spark8, 1, 11, Finished, Available)

In [ ]:
# Display for debugging

registeredModel

StatementMeta(spark8, 1, 12, Finished, Available)

In [ ]:
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = cpu_cores, 
    memory_gb = memory_gb, 
    tags = tags, 
    description = description)

StatementMeta(spark8, 1, 13, Finished, Available)

In [ ]:
conda_env = CondaDependencies.create(conda_packages = conda_packages, pip_packages = pip_packages)

with open(conda_file, "w") as f:
    f.write(conda_env.serialize_to_string())

StatementMeta(spark8, 1, 14, Finished, Available)

In [ ]:
myenv = Environment.from_conda_specification(name = env_name, file_path = conda_file)

myenv.register(workspace = ws)

StatementMeta(spark8, 1, 15, Finished, Available)

In [ ]:
inference_config = InferenceConfig(entry_script = scoring_file, environment = myenv)

StatementMeta(spark8, 1, 16, Finished, Available)

In [ ]:
# Display for debugging

inference_config

StatementMeta(spark8, 1, 17, Finished, Available)

In [ ]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

StatementMeta(spark8, 1, 18, Finished, Available)

In [ ]:
# Display for debugging

deployment_config

StatementMeta(spark8, 1, 19, Finished, Available)

In [18]:
service = Model.deploy(ws, service_name, [registeredModel], inference_config, deployment_config, overwrite = True)

StatementMeta(spark8, 1, 20, Finished, Available)

In [19]:
service.wait_for_deployment(show_output = True)

StatementMeta(spark8, 1, 21, Finished, Available)

Running.............................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"

In [20]:
print(service.state)

StatementMeta(spark8, 1, 22, Finished, Available)

Healthy

In [21]:
print(service.get_logs())

StatementMeta(spark8, 1, 23, Finished, Available)

2020-08-17T23:11:11,579213876+00:00 - gunicorn/run 
2020-08-17T23:11:11,580839185+00:00 - iot-server/run 
2020-08-17T23:11:11,582128092+00:00 - rsyslog/run 
2020-08-17T23:11:11,584529506+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_217d8387ec6ddcca5d8194ee91c500fc/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_217d8387ec6ddcca5d8194ee91c500fc/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_217d8387ec6ddcca5d8194ee91c500fc/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_217d8387ec6ddcca5d8194ee91c500fc/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_217d8387ec6ddcca5d8194ee91c500fc/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC